In [ ]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any

import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch import Tensor
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig
from transformers import pipeline, AutoTokenizer

model_name = "google/mt5-small"
base_model = models.Transformer(model_name)
# sentence bert repo does not current support mT5 yet (only T5 ironically)
MT5EncoderModel._keys_to_ignore_on_load_unexpected = ["decoder.*"]
mT5_config = AutoConfig.from_pretrained(model_name)   
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import pandas as pd
train_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/train.csv')
test_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/test.csv')

In [ ]:
train_df['Text']=train_df['Text'].tolist()
test_df['Text']=test_df['Text'].tolist()
train_batch_sentences=[]
test_batch_sentences=[]

for frame in train_df['Text']:
    train_batch_sentences.append(frame)
for sentence in test_df['Text']:
    test_batch_sentences.append(frame)

In [ ]:
train_batch_sentences

In [ ]:
import pickle
train_representations=tokenizer(train_batch_sentences,padding=True,truncation=True)
test_representations=tokenizer(test_batch_sentences,padding=True,truncation=True)

train_filehandler = open("train.pkl","wb")
test_filehandler = open("test.pkl","wb")
pickle.dump(train_representations,train_filehandler)
pickle.dump(test_representations,test_filehandler)
train_filehandler.close()
test_filehandler.close()